## Loading sql and the Data Warehouse

In [1]:
%load_ext sql
%sql sqlite:///CourseDataWarehouse.db

'Connected: @CourseDataWarehouse.db'

In [2]:
%%sql
attach "CourseData.db" as reldb

 * sqlite:///CourseDataWarehouse.db
Done.


[]

### Dropping tables to avoid duplicates

In [3]:
%%sql 
DROP TABLE IF EXISTS PROGRAMS_WH;
DROP TABLE IF EXISTS LOCATIONS_WH;
DROP TABLE IF EXISTS TIMES_WH;
DROP TABLE IF EXISTS FACULTY_WH;
DROP TABLE IF EXISTS CATALOGS_WH;
DROP TABLE IF EXISTS COURSES_WH;

 * sqlite:///CourseDataWarehouse.db
Done.
Done.
Done.
Done.
Done.
Done.


[]

## Creating the Dimension tables for the Star Schema

In [4]:
%%sql
CREATE TABLE PROGRAMS_WH (
    PID INTEGER NOT NULL PRIMARY KEY,
    program_name TEXT NOT NULL,
    program_code VARCHAR NOT NULL
);

 * sqlite:///CourseDataWarehouse.db
Done.


[]

In [5]:
%%sql
CREATE TABLE LOCATIONS_WH (
    LID INTEGER PRIMARY KEY,
    LName VARCHAR
);

 * sqlite:///CourseDataWarehouse.db
Done.


[]

In [6]:
%%sql 
CREATE TABLE TIMES_WH (
    TID INTEGER NOT NULL PRIMARY KEY,
    Day TEXT NOT NULL,
    Start TEXT NOT NULL,
    End TEXT NOT NULL,   
    Term TEXT NOT NULL
);

 * sqlite:///CourseDataWarehouse.db
Done.


[]

In [7]:
%%sql 
CREATE TABLE FACULTY_WH (
    FID  INTEGER NOT NULL PRIMARY KEY,
    FName TEXT NOT NULL
);

 * sqlite:///CourseDataWarehouse.db
Done.


[]

In [8]:
%%sql
CREATE TABLE CATALOGS_WH (
    CatID INTEGER PRIMARY KEY,
    CourseCode VARCHAR NOT NULL,
    Credits TEXT NOT NULL,
    Prereqs TEXT,
    Coreqs TEXT,
    Fees INTEGER,
    Description TEXT,
    Attributes TEXT
);

 * sqlite:///CourseDataWarehouse.db
Done.


[]

### Creating the fact table of the Star Schema (COURSES)

In [9]:
%%sql
CREATE TABLE COURSES_WH (
    CID INTEGER PRIMARY KEY,
    CRN INTEGER NOT NULL,
    Title TEXT NOT NULL,
    Section VARCHAR NOT NULL,
    Act INTEGER,
    Cap INTEGER,
    Rem INTEGER, 
    AvgCap INTEGER,
    NumClasses INTEGER,
    PID INTEGER NOT NULL,
    LID INTEGER NOT NULL,
    TID INTEGER NOT NULL,
    FID INTEGER NOT NULL,
    CatID INTEGER NOT NULL,
    FOREIGN KEY (PID) REFERENCES PROGRAMS (PID),
    FOREIGN KEY (LID) REFERENCES LOCATIONS (LID),
    FOREIGN KEY (TID) REFERENCES TIMES (TID),
    FOREIGN KEY (FID) REFERENCES FACULTY (FID),
    FOREIGN KEY (CatID) REFERENCES CATALOGS (CatID)
);

 * sqlite:///CourseDataWarehouse.db
Done.


[]

### 

In [10]:
%%sql

INSERT INTO PROGRAMS_WH (program_name, program_code)
SELECT DISTINCT program_name, program_code
FROM reldb.PROGRAMS;

 * sqlite:///CourseDataWarehouse.db
83 rows affected.


[]

In [11]:
%%sql

INSERT INTO LOCATIONS_WH (LName)
SELECT DISTINCT location
FROM reldb.COURSE_MEETINGS;

 * sqlite:///CourseDataWarehouse.db
201 rows affected.


[]

In [12]:
%%sql

INSERT INTO TIMES_WH (Day, Start, End, Term)
SELECT DISTINCT day, Start, End, term
FROM reldb.COURSE_MEETINGS;

 * sqlite:///CourseDataWarehouse.db
41838 rows affected.


[]

In [13]:
%%sql

INSERT INTO FACULTY_WH (FName)
SELECT DISTINCT FName
FROM reldb.FACULTY;

 * sqlite:///CourseDataWarehouse.db
1104 rows affected.


[]

In [14]:
%%sql

INSERT INTO CATALOGS_WH (CourseCode, credits, Prereqs, Coreqs, Fees, Description, Attributes)
SELECT DISTINCT CourseCode, credits, prereqs, coreqs, fees, description, attributes
FROM reldb.CATALOGS;

 * sqlite:///CourseDataWarehouse.db
2220 rows affected.


[]

In [18]:
%%sql

INSERT INTO COURSES_WH (CRN, Title, Section, Act, Cap, Rem, PID, LID, TID, FID, CatID)
SELECT FROM DISTINCT CRN, title, section, act, cap, rem, PID, LID, TID, FID, CatID
FROM reldb.COURSES;

 * sqlite:///CourseDataWarehouse.db
(sqlite3.OperationalError) near "FROM": syntax error [SQL: 'INSERT INTO COURSES_WH (CRN, Title, Section, Act, Cap, Rem, CatID)\nSELECT FROM DISTINCT CRN, title, section, act, cap, rem, CatID\nFROM reldb.COURSES\n    JOIN reldb.CATALOGS ON (title);'] (Background on this error at: http://sqlalche.me/e/e3q8)
